# Given example and template, demonstrate how chat adapter works

In [1]:
%load_ext autoreload
%autoreload 2

import openai
import os

openai.api_key = os.getenv('PERSONAL_OPENAI_API_KEY')
openai.api_base = os.getenv('API_BASE')

In [2]:
try: import google.colab; root_path = 'dsp'
except: root_path = '.'
    
import dsp
    
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

openai_key = os.getenv('PERSONAL_OPENAI_API_KEY')  # or replace with your API key (optional)
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'

Not loading Cohere because it is not installed.


In [3]:
train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

# Train is an array of examples
train = [dsp.Example(question=question, answer=answer) for question, answer in train]

In [4]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']),
       ('What documentary about the Gilgo Beach Killer debuted on A&E?', ['The Killing Season']),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', ['Gainesville, Florida', 'Gainesville']),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', ['1828']),
       ('Which author is English: John Braine or Studs Terkel?', ['John Braine']),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', ['Andre Dirrell']),
       ('In which city is the sports nutrition business established by Oliver Cookson based ?', ['Cheshire', 'Cheshire, UK']),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', ['February 13, 1980']),
       ('Kyle Moran was born in the town on what river?', ['Castletown', 'Castletown River']),
       ("What is the name of one branch of Robert D. Braun's speciality?", ['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']),
       ("Where was the actress who played the niece in the Priest film born?", ['Surrey', 'Guildford, Surrey']),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', ['Portrait of a Marriage']),
       ('What year was the father of the Princes in the Tower born?', ['1442'])]

dev = [dsp.Example(question=question, answer=answer) for question, answer in dev]

In [5]:
# Template, Demos, and Question Construction
Question = dsp.Type(prefix="Question:", desc="${the question to be answered}")
Answer = dsp.Type(prefix="Answer:", desc="${a short factoid answer, often between 1 and 5 words}", format=dsp.format_answers)
qa_template = dsp.Template(instructions="Answer questions with short factoid answers.", question=Question(), answer=Answer())

### Load Models and Adapters

In [95]:
from dsp.evaluation.utils import evaluate
from dsp.modules import ChatAdapter, BasicAdapter

rm = dsp.ColBERTv2(url=colbert_server)
davinci = dsp.GPT3(model='text-davinci-002', api_key=openai_key)
turbo = dsp.GPT3(model='gpt-3.5-turbo-0613', api_key=openai_key, model_type="chat")

# Program 1: Vanilla GPT-3.5

In [22]:
def vanilla_LM_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    example = dsp.Example(question=question, demos=demos)
    example, completions = dsp.generate(qa_template)(example, stage='qa')
    return completions.answer

## Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [23]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=False, multi_turn=False, strict_turn=False))
vanilla_LM_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\nQuestion: What documentary about the Gilgo Beach Killer debuted on A&E?\nAnswer:'}]}The Killing Season





('The Killing Season', None)

In [24]:
evaluate(vanilla_LM_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 504.83it/s]

Answered 4 / 13 (30.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1971,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",1947,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Mississippi River,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Space Systems Engineering,❌


30.8

## Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [25]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=False, strict_turn=False))
vanilla_LM_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\nQuestion: What documentary about the Gilgo Beach Killer debuted on A&E?\nAnswer:'}]}The Killin

('The Killing Season', None)

In [26]:
evaluate(vanilla_LM_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 723.75it/s]

Answered 5 / 13 (38.5%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1964,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']",Manchester,❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","September 19, 1940",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Mississippi River,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Space Systems Engineering,❌


38.5

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [27]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=False))
vanilla_LM_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'user', 'content': 'Question: What documentary about the Gilgo 

('The Killing Season', None)

In [28]:
evaluate(vanilla_LM_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 720.62it/s]

Answered 4 / 13 (30.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Ocala, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1971,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],David Benavidez,❌
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, United Kingdom",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",No information provided.,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Unknown.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace engineering,❌


30.8

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [29]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=True))
vanilla_LM_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'assistant', 'cont

('The Killing Season', None)

In [30]:
evaluate(vanilla_LM_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 677.19it/s]

Answered 4 / 13 (30.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1971,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, United Kingdom",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","Sorry, but I can't provide the information you're looking for.",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']","Sorry, but I can't provide the answer to that question.",❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Space systems engineering,❌


30.8

# Program 2: Retrieve-then-Read w/ GPT-3.5

In [31]:
Context = dsp.Type(
    prefix="Context:\n",
    desc="${sources that may contain relevant content}",
    format=dsp.passages2text
)

qa_template_with_passages = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), answer=Answer()
)

In [32]:
def retrieve_then_read_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=1)
    
    example = dsp.Example(question=question, context=passages, demos=demos)
    example, completions = dsp.generate(qa_template_with_passages)(example, stage='qa')

    return completions.answer

## Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [33]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=False, multi_turn=False, strict_turn=False))
retrieve_then_read_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nContext:\n«Joel Rifkin | to the Gilgo Beach Killer.

('The Killing Season', None)

In [34]:
evaluate(retrieve_then_read_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 409.09it/s]

Answered 4 / 13 (30.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],No relevant information provided.,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']",UK,❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",5 August 1862,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Jet Propulsion Section,❌


30.8

## Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [35]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=False, strict_turn=False))
retrieve_then_read_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nContext:\n«Joel Rifkin | to

('The documentary is called "The Long Island Serial Killer: A Mother\'s Hunt for Justice."',
 None)

In [36]:
evaluate(retrieve_then_read_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 673.64it/s]

Answered 4 / 13 (30.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"The documentary is called ""The Long Island Serial Killer: A Mother's Hunt for Justice.""",❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],No relevant information provided.,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']",UK,❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",No relevant content found.,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Jet Propulsion Section,❌


30.8

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [37]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=False))
retrieve_then_read_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'user', 'content': 'Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}, {'role': 'us

('The documentary about the Gilgo Beach Killer that debuted on A&E is called "The Killing Season."',
 None)

In [38]:
evaluate(retrieve_then_read_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 559.76it/s]

Answered 3 / 13 (23.1%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"The documentary about the Gilgo Beach Killer that debuted on A&E is called ""The Killing Season.""",❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"Answer: The party of the winner of the 1971 San Francisco mayoral election, Joseph Alioto, was founded in 1854.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']",Manchester,❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","Birth date: August 17, 1943",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",The factoid answer is not provided in the given context.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace Engineering,❌


23.1

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [39]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=True))
retrieve_then_read_QA(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': 'Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nAnswer: ${a short factoid answer,

('The documentary is called "The Killing Season."', None)

In [40]:
evaluate(retrieve_then_read_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 611.33it/s]

Answered 3 / 13 (23.1%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"The documentary is called ""The Killing Season.""",❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Answer: The singer was born in Ocala, Florida.",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"Answer: The party of the winner of the 1971 San Francisco mayoral election, Joseph Alioto, was founded in 1854.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","The sports nutrition business established by Oliver Cookson, Myprotein, is based in the UK.",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","I'm sorry, but I couldn't find any relevant information about the birth date of the actor who played roles in First Wives Club and Searching for the Elephant based on the provided context.",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",The answer is not provided in the given context.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace Engineering,❌


23.1

# Program 3: Retrieve-then-Read w/ Self-Consistency

In [41]:
Rationale = dsp.Type(
    prefix="Rationale: Let's think step by step.",
    desc="${a step-by-step deduction that identifies the correct response, which will be provided below}"
)

qa_template_with_CoT = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), rationale=Rationale(), answer=Answer()
)

In [42]:
@dsp.transformation
def QA_predict(example: dsp.Example, sc=False):
    if sc:
        example, completions = dsp.generate(qa_template_with_CoT, n=20, temperature=0.7)(example, stage='qa')
        completions = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_with_CoT)(example, stage='qa')
    
    return example.copy(answer=completions.answer)

def retrieve_then_read_QA_v2(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=5)
    example = dsp.Example(question=question, context=passages, demos=demos)
    
    return QA_predict(example).answer

## Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [43]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=False, multi_turn=False, strict_turn=False))
retrieve_then_read_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct response, which will be provide

('The Killing Season', None)

In [44]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 329.03it/s]

Answered 7 / 13 (53.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],Democratic Party,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",The birth date of the actor is not provided in the given sources.,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Unknown,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


53.8

## Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [45]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=False, strict_turn=False))
retrieve_then_read_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct respons

('"The Killing Season"', None)

In [46]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 497.95it/s]

Answered 6 / 13 (46.2%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']","It is not explicitly mentioned which author, Julia Peterkin or E. L. Doctorow, has a broader scope of profession.",❌
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"""The Killing Season""",✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"The party of the winner of the 1971 San Francisco mayoral election, Joseph Alioto, was founded in 1854.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","March 15, 1973",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']","The town of Dundalk, where Kyle Moran was born, is located on the River Castletown.",❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


46.2

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [47]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=False))
retrieve_then_read_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by step. ${a step-by-step deduction that identifie

('The Killing Season', None)

In [48]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 451.01it/s]

Answered 6 / 13 (46.2%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",broader scope of profession.,❌
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],Democratic Party,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","The birth date of the actor who played roles in ""First Wives Club"" and ""Searching for the Elephant"" is unknown.",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk is located on the Castletown River.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration.,❌


46.2

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [49]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=True))
retrieve_then_read_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by st

('"The Killing Season"', None)

In [50]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 588.58it/s]

Answered 6 / 13 (46.2%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']","E. L. Doctorow has a broader scope of profession as he is known for his works of historical fiction, while Julia Peterkin is primarily known for her novels about the plantation South and the African-American experience.",❌
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"""The Killing Season""",✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Ocala,❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],Democratic Party,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",Peter McEnery,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",N/A (The town on the river is not mentioned in the provided context),❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration.,❌


46.2

# Program 4: Multi-Hop Retrieval

In [85]:
SearchRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. To answer this question, we first need to find out",
    desc="${the missing information}"
)

SearchQuery = dsp.Type(
    prefix="Search Query:",
    desc="${a simple question for seeking the missing information}"
)

rewrite_template = dsp.Template(
    instructions="Write a search query that will help answer a complex question.",
    question=Question(), rationale=SearchRationale(), query=SearchQuery()
)

In [86]:
CondenseRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. Based on the context, we have learned the following.",
    desc="${information from the context that provides useful clues}"
)

hop_template = dsp.Template(
    instructions=rewrite_template.instructions,
    context=Context(), question=Question(), rationale=CondenseRationale(), query=SearchQuery()
)

In [87]:
from dsp.utils import deduplicate

@dsp.transformation
def multihop_search_v1(example: dsp.Example, max_hops=2, k=2) -> dsp.Example:
    example.context = []
    
    for hop in range(max_hops):
        # Generate a query based
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template)(example, stage=f'h{hop}')

        # Retrieve k results based on the query generated
        passages = dsp.retrieve(completions.query, k=k)

        # Update the context by concatenating old and new passages
        example.context = deduplicate(example.context + passages)

    return example


def multihop_QA_v1(question: str) -> str:
    demos = dsp.sample(train, k=7)
    x = dsp.Example(question=question, demos=demos)
    
    x = multihop_search_v1(x)
    x = QA_predict(x, sc=False)

    return x.answer

## Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [88]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=False, multi_turn=False, strict_turn=False))
multihop_QA_v1(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct response, which will be provide

('The Menendez Murders: Erik Tells All', None)

In [89]:
evaluate(multihop_QA_v1, dev)

100%|██████████| 13/13 [00:00<00:00, 1358.70it/s]

Answered 8 / 13 (61.5%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Menendez Murders: Erik Tells All,❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']",Gainesville,✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"The party of Joseph Alioto, the winner of the 1971 San Francisco mayoral election, was not mentioned in the provided context.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","November 4, 1968",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Castletown River,✔️
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


61.5

## Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [90]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=False, strict_turn=False))
multihop_QA_v1(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct respons

('"The Menendez Murders: Erik Tells All"', None)

In [91]:
evaluate(multihop_QA_v1, dev)

  0%|          | 0/13 [00:10<?, ?it/s]


ConnectTimeout: HTTPConnectionPool(host='ec2-44-228-128-229.us-west-2.compute.amazonaws.com', port=8893): Max retries exceeded with url: /api/search?query=What+are+the+professions+of+E.+L.+Doctorow+and+Julia+Peterkin%3F&k=2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x126a4cf50>, 'Connection to ec2-44-228-128-229.us-west-2.compute.amazonaws.com timed out. (connect timeout=10)'))

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [61]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=False))
multihop_QA_v1(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by step. ${a step-by-step deduction that identifie

('There is no information provided about a documentary about the Gilgo Beach Killer that debuted on A&E.',
 None)

In [62]:
evaluate(multihop_QA_v1, dev)

  0%|          | 0/13 [00:12<?, ?it/s]


ConnectTimeout: HTTPConnectionPool(host='ec2-44-228-128-229.us-west-2.compute.amazonaws.com', port=8893): Max retries exceeded with url: /api/search?query=What+professions+did+E.+L.+Doctorow+and+Julia+Peterkin+have%3F&k=2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x125ff5990>, 'Connection to ec2-44-228-128-229.us-west-2.compute.amazonaws.com timed out. (connect timeout=10)'))

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [ ]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=True))
multihop_QA_v1(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by st

('"The Menendez Murders: Erik Tells All"', None)

In [ ]:
evaluate(multihop_QA_v1, dev)

100%|██████████| 13/13 [00:00<00:00, 3445.56it/s]

Answered 3 / 13 (23.1%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],"The documentary is called ""The Killing Season.""",❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Answer: The singer was born in Ocala, Florida.",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"Answer: The party of the winner of the 1971 San Francisco mayoral election, Joseph Alioto, was founded in 1854.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","The sports nutrition business established by Oliver Cookson, Myprotein, is based in the UK.",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","I'm sorry, but I couldn't find any relevant information about the birth date of the actor who played roles in First Wives Club and Searching for the Elephant based on the provided context.",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",The answer is not provided in the given context.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace Engineering,❌


23.1

# Program 5 with Turbo: Multi-Hop Condensed Retrieval w/ Automatic Demos and Query Fusion

In [63]:
@dsp.transformation
def multihop_attempt(d: dsp.Example) -> dsp.Example:
    # Prepare unaugmented demonstrations for the example.
    x = dsp.Example(question=d.question, demos=dsp.all_but(train, d))
    
    # Search. And skip examples where search fails.
    # Annotate demonstrations for multihop_search_v2 with the simpler multihop_search_v1 pipeline.
    x = multihop_search_v1(x)
    if not dsp.passage_match(x.context, d.answer): return None
    
    # Predict. And skip examples where predict fails.
    x = QA_predict(x, sc=False)
    if not dsp.answer_match(x.answer, d.answer): return None
    
    return d.copy(**x)

@dsp.transformation
def multihop_demonstrate(x: dsp.Example) -> dsp.Example:
    demos = dsp.sample(train, k=7)
    x.demos = dsp.annotate(multihop_attempt)(demos, k=3, return_all=True)
    return x

In [64]:
@dsp.transformation
def multihop_search_v2(example: dsp.Example, max_hops=2, k=5) -> dsp.Example:
    example.context = []

    for hop in range(max_hops):
        # Generate queries
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template, n=10, temperature=0.7)(example, stage=f'h{hop}')
        
        # Collect the queries and search with result fusion
        queries = [c.query for c in completions] + [example.question]
        example.context = dsp.retrieveEnsemble(queries, k=k)

        # Arrange the passages for the next hop
        if hop > 0:
            example.context = [completions[0].rationale] + example.context
    
    return example

def multihop_QA_v2(question: str) -> str:
    x = dsp.Example(question=question)
    x = multihop_demonstrate(x)
    x = multihop_search_v2(x)
    x = QA_predict(x)
    return x.answer

## Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [65]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=False, multi_turn=False, strict_turn=False))
multihop_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nContext:\n[1] «Gary Zukav | experience that led to his first book, "Dancing Wu Li Masters", written with extensive help from Jack Sarfatti and other physicists he met through Sarfatti as described in David Kaiser\'s book "How the Hippies Save

('The Killing Season', None)

In [66]:
evaluate(multihop_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 43.61it/s]

Answered 10 / 13 (76.9%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"The party of the winner, Joseph Alioto, was already established before the 1971 election.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",1958,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Castletown River,✔️
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


76.9

## Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [67]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=False, strict_turn=False))
multihop_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nFollow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let\'s think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nContext:\n[1] «Gary Zukav | this book enjoyable and informative." David Bohm, renowned quantum physicist, wrote a personal endorsement provided to the book\'s publisher Harper Collins: "Recommended highly for those who

('The Killing Season', None)

In [68]:
evaluate(multihop_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 49.65it/s]

Answered 11 / 13 (84.6%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1828,✔️
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","February 13, 1980",✔️
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Castletown River,✔️
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


84.6

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [69]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=False))
multihop_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}"}, {'role': 'user', 'content': 'Context:\n[1] «Gary Zukav | experience that led to his first book, "Dancing Wu Li Masters", written with extensive help from Jack S

('The Killing Season', None)

In [70]:
evaluate(multihop_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 43.80it/s]

Answered 10 / 13 (76.9%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],The context does not provide information about the year the party of the winner of the 1971 San Francisco mayoral election was founded.,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","February 13, 1980",✔️
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Castletown River,✔️
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration,❌


76.9

## Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [71]:
dsp.settings.configure(lm=turbo, rm=rm, adapter=ChatAdapter(system_turn=True, multi_turn=True, strict_turn=True))
multihop_QA_v2(dev[1].question), turbo.inspect_history(n=1)





{'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'system', 'content': 'Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: What city was the victim of Joseph Druces working in?\nAnswer: Boston, Massachusetts\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925'}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': "Follow the following format.\n\nContext:\n${sources that may contain relevant content}\n\nQuestion: ${the question to be answered}\n\nRationale: Let's think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}\n\nAnswer: ${a short factoid answer, often between 1 and 5 words}"}, {'role': 'assistant', 'content': 'Got it.'}, {'role': 'user', 'content': 'Context:\n[1] «Gary Zukav | Gary Zukav Gary Zukav (born Octo

('The Killing Season (U.S. TV series)', None)

In [72]:
evaluate(multihop_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 45.02it/s]

Answered 5 / 13 (38.5%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season (U.S. TV series),❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],"The party of the winner, Joseph Alioto, was the Democratic Party, which was founded in 1828.",❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine is English.,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK.",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","Lee Joon-hyuk was born on March 13, 1984.",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Not enough information provided to determine the town on the river where Kyle Moran was born.,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Planetary exploration.,❌


38.5

# Program 5 with Davinci: Multi-Hop Condensed Retrieval w/ Automatic Demos and Query Fusion

In [96]:
@dsp.transformation
def multihop_attempt(d: dsp.Example) -> dsp.Example:
    # Prepare unaugmented demonstrations for the example.
    x = dsp.Example(question=d.question, demos=dsp.all_but(train, d))
    
    # Search. And skip examples where search fails.
    # Annotate demonstrations for multihop_search_v2 with the simpler multihop_search_v1 pipeline.
    x = multihop_search_v1(x)
    if not dsp.passage_match(x.context, d.answer): return None
    
    # Predict. And skip examples where predict fails.
    x = QA_predict(x, sc=False)
    if not dsp.answer_match(x.answer, d.answer): return None
    
    return d.copy(**x)

@dsp.transformation
def multihop_demonstrate(x: dsp.Example) -> dsp.Example:
    demos = dsp.sample(train, k=7)
    x.demos = dsp.annotate(multihop_attempt)(demos, k=3, return_all=True)
    return x

In [97]:
@dsp.transformation
def multihop_search_v3(example: dsp.Example, max_hops=2, k=5) -> dsp.Example:
    example.context = []

    for hop in range(max_hops):
        # Generate queries
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template, n=10, temperature=0.7)(example, stage=f'h{hop}')
        
        # Collect the queries and search with result fusion
        queries = [c.query for c in completions] + [example.question]
        example.context = dsp.retrieveEnsemble(queries, k=k)

        # Arrange the passages for the next hop
        if hop > 0:
            example.context = [completions[0].rationale] + example.context
    
    return example

def multihop_QA_v3(question: str) -> str:
    x = dsp.Example(question=question)
    with dsp.settings.context(lm=davinci):
        x = multihop_demonstrate(x)
    x = multihop_search_v3(x)
    x = QA_predict(x)
    return x.answer

## Basic Adapter

In [98]:
dsp.settings.configure(lm=davinci, rm=rm, adapter=BasicAdapter())
multihop_QA_v3(dev[1].question), davinci.inspect_history(n=1)

{'temperature': 0.0, 'max_tokens': 150, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'model': 'text-davinci-002', 'prompt': "Write a search query that will help answer a complex question.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nRationale: Let's think step by step. To answer this question, we first need to find out ${the missing information}\nSearch Query: ${a simple question for seeking the missing information}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nRationale: Let's think step by step. To answer this question, we first need to find out"}


ConnectTimeout: HTTPConnectionPool(host='ec2-44-228-128-229.us-west-2.compute.amazonaws.com', port=8893): Max retries exceeded with url: /api/search?query=%22What+is+the+name+of+Gary+Zukav%27s+first+book%3F%22&k=2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x126df8dd0>, 'Connection to ec2-44-228-128-229.us-west-2.compute.amazonaws.com timed out. (connect timeout=10)'))

In [82]:
evaluate(multihop_QA_v3, dev)

  0%|          | 0/13 [00:00<?, ?it/s]


TypeError: unhashable type: 'dict'